In [ ]:
"""
Adapted from “Cerebral chemoarchitecture shares organizational traits with brain structure and function” [Hänisch et al. eLife 2023] 

by Carlos Estevez-Fraga
"""

In [ ]:
from scipy.stats import zscore
import nilearn
import numpy as np
from nilearn import datasets, plotting
from nilearn.input_data import NiftiMasker
from palettable.colorbrewer.qualitative import Set2_4, Set2_6
import os
import pandas as pd
from brainspace.datasets import load_parcellation, load_conte69
from brainspace.gradient import GradientMaps
from brainspace.plotting import plot_hemispheres
from brainspace.utils.parcellation import map_to_labels
import seaborn as sns
import matplotlib.pyplot as plt
import ptitprince as pt
from mn_funcs import spearman, vgm
from brainspace.null_models import SurrogateMaps
from palettable.scientific.diverging import Roma_4
from netneurotools import datasets, stats
from statsmodels.stats.multitest import multipletests

In [ ]:
input_path = '/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/data/'
res_p = '/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/F2/'
mask_p='/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/data/subcortical_masks/'

# load data
mask = NiftiMasker(
    mask_p + 'tian_binary_mask_total.nii.gz').fit()

# hierarchical clustering
subcortex_regions = pd.read_csv(input_path + 'subcortex_tian_regions.csv', index_col=0)
sns.set(font_scale=2.3)
sns.clustermap(subcortex_regions, method='weighted', cmap='coolwarm', figsize=(15, 10), vmin=-2.4, vmax=2.4,
               tree_kws={'linewidths': 2})

# subcortical gradients
# make and subcortical display
#subcortex = pd.read_csv(input_path + 'tian_subcortex.csv', index_col=0)
subcortex = pd.read_csv(input_path + 'subcortex_tian.csv', index_col=0)
subcortex = subcortex.apply(zscore)
sc_corr = subcortex.transpose().corr('spearman')
gm = GradientMaps(n_components=10, approach='dm', kernel='normalized_angle')
gm.fit(sc_corr.values)
s_g1 = gm.gradients_[:, 0]
s_g2 = gm.gradients_[:, 1]
s_g3 = gm.gradients_[:, 2]


#Save gradients
np.save(input_path + 's_g1.npy', s_g1)
np.savetxt(input_path + 's_g1.csv', s_g1, delimiter=',')

np.save(input_path + 's_g2.npy', s_g2)
np.savetxt(input_path + 's_g2.csv', s_g2, delimiter=',')

np.save(input_path + 's_g3.npy', s_g3)
np.savetxt(input_path + 's_g3.csv', s_g3, delimiter=',')


In [ ]:
np.save(input_path + 's_g1.npy', s_g1)
np.savetxt(input_path + 's_g1.csv', s_g1, delimiter=',')

np.save(input_path + 's_g2.npy', s_g2)
np.savetxt(input_path + 's_g2.csv', s_g2, delimiter=',')

np.save(input_path + 's_g3.npy', s_g3)
np.savetxt(input_path + 's_g3.csv', s_g3, delimiter=',')


In [ ]:
#Save gradient 1

np.save(input_path + 'g1_vgm_f.npy', g1_vgm_f)
np.savetxt(input_path + 'g1_vgm_f.csv', g1_vgm_f, delimiter=',')

In [ ]:
#gradient-receptor-correlations
r_1={}
for i in subcortex.columns:
    sub=subcortex[i]
    r_1[i]=vgm(sub, s_g1, g1_vgm_f)

In [ ]:
def plot_dens_corr(inp, fname):
    df1=pd.DataFrame.from_dict(inp, orient='index')
    df1.columns=["Spearman's r", 'p']
    df1.sort_values(by="Spearman's r", inplace=True)
    fig, ax=plt.subplots(figsize=(15,5))
    color=['lightskyblue' if x > 0.05 else 'dodgerblue' for x in df1['p']]
    ax.bar(range(len(df1)), df1["Spearman's r"], color=color)
    ax.set_xticks(range(len(df1)), labels=list(df1.index))
    ax.set_ylabel("Spearman's r", fontsize=28)
    ax.tick_params(labelsize=26)
    plt.xticks(rotation=30+270)
    plt.tight_layout()
    fig.savefig(res_p + fname)
    return

plot_dens_corr(r_1, 'G1_receptors.png')


In [ ]:
# scree
scat = [(x / sum(gm.lambdas_)) * 100 for x in gm.lambdas_]
fig, ax = plt.subplots(figsize=(8, 8))
sns.set_style('ticks')
ax.scatter(range(1, len(scat) + 1), scat, s=140)
ax.plot(range(1, len(scat) + 1), scat, '--')
ax.set_xlabel('# component', fontsize=28)
ax.set_xticks(range(1, 11))
ax.set_ylabel('% variance explained', fontsize=28)
ax.tick_params(labelsize=24)
plt.tight_layout()
sns.despine()
fig.savefig(res_p + 'sc_grad_scree.png')

In [ ]:
# subcortico-subcortical projection
# using voxelwise differences from the SPM MAPS


#Transform gradients into DFs
s_g1_df = pd.DataFrame(s_g1, columns=["Value"])
s_g2_df = pd.DataFrame(s_g2, columns=["Value"])
s_g3_df = pd.DataFrame(s_g3, columns=["Value"])


#Obtain voxelwise volumetric data
track_sxc=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/data/subcortex_masked/track_spmT_0002_tian.csv', header=None)
track_sxc=track_sxc.transpose()


trackon_sxc=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/data/subcortex_masked/trackon_spmT_0002_tian.csv', header=None)
trackon_sxc=trackon_sxc.transpose()

paddington_sxc=pd.read_csv('/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/data/subcortex_masked/paddington_spmT_0002_tian.csv', header=None)
paddington_sxc=paddington_sxc.transpose()



In [ ]:
# Plain correlations
#Correlation track-gradients
sxc_1,p1 = spearmanr(track_sxc.values, s_g1_df.values)
sxc_2,p2 = spearmanr(track_sxc.values, s_g2_df.values)
sxc_3,p3 = spearmanr(track_sxc.values, s_g3_df.values)

correlation_data = {
    "p1": p1,
    "sxc_1": sxc_1,
    "p2": p2,
    "sxc_2": sxc_2,
    "p3": p3,
    "sxc_3": sxc_3
}

output_file = "/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/sxc_track.txt"

header_text = "Correlation coefficients/Pvals in Track between voxelwise subcortical volumes subcortical receptomes 1, 2 and 3 (in order).\n"

# Save the correlation_data dictionary to the text file
with open(output_file, 'w') as file:
    file.write(header_text)
    json.dump(correlation_data, file)

print(sxc_1)
print(sxc_2)
print(sxc_3)

In [ ]:
# Plain correlations
#Correlation trackon-gradients
sxc_1,p1 = spearmanr(trackon_sxc.values, s_g1_df.values)
sxc_2,p2 = spearmanr(trackon_sxc.values, s_g2_df.values)
sxc_3,p3 = spearmanr(trackon_sxc.values, s_g3_df.values)

correlation_data = {
    "p1": p1,
    "sxc_1": sxc_1,
    "p2": p2,
    "sxc_2": sxc_2,
    "p3": p3,
    "sxc_3": sxc_3
}

output_file = "/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/sxc_trackon.txt"

header_text = "Correlation coefficients/Pvals in Trackon between voxelwise subcortical volumes subcortical receptomes 1, 2 and 3 (in order).\n"

# Save the correlation_data dictionary to the text file
with open(output_file, 'w') as file:
    file.write(header_text)
    json.dump(correlation_data, file)

print(sxc_1)
print(sxc_2)
print(sxc_3)

In [ ]:
#Now the same but with permutations
#Correlation track-gradients
sxc_1,p1 = stats.permtest_pearsonr(track_sxc.values, s_g1_df.values)
sxc_2,p2 = stats.permtest_pearsonr(track_sxc.values, s_g2_df.values)
sxc_3,p3 = stats.permtest_pearsonr(track_sxc.values, s_g3_df.values)

p_values = [p1, p2, p3]

# Perform FDR correction
corrected_p_values = multipletests(p_values, method='fdr_bh')[1]

# Assign the corrected p-values back to p1, p2, and p3
p1, p2, p3 = corrected_p_values

correlation_data = {
    "p1": p1,
    "sxc_1": sxc_1,
    "p2": p2,
    "sxc_2": sxc_2,
    "p3": p3,
    "sxc_3": sxc_3
}

output_file = "/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/permut_sxc_track.txt"

header_text = "Permuted correlation coefficients/Pvals in Track between voxelwise subcortical volumes subcortical receptomes 1, 2 and 3 (in order).\n"

# Save the correlation_data dictionary to the text file
with open(output_file, 'w') as file:
    file.write(header_text)
    json.dump(correlation_data, file)

print(sxc_1)
print(sxc_2)
print(sxc_3)


#Correlation trackon-gradients
sxc_1,p1 = stats.permtest_pearsonr(trackon_sxc.values, s_g1_df.values)
sxc_2,p2 = stats.permtest_pearsonr(trackon_sxc.values, s_g2_df.values)
sxc_3,p3 = stats.permtest_pearsonr(trackon_sxc.values, s_g3_df.values)

p_values = [p1, p2, p3]

# Perform FDR correction
corrected_p_values = multipletests(p_values, method='fdr_bh')[1]

# Assign the corrected p-values back to p1, p2, and p3
p1, p2, p3 = corrected_p_values

correlation_data = {
    "p1": p1,
    "sxc_1": sxc_1,
    "p2": p2,
    "sxc_2": sxc_2,
    "p3": p3,
    "sxc_3": sxc_3
}

output_file = "/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/permut_sxc_trackon.txt"

header_text = "Permuted correlation coefficients/Pvals in Track between voxelwise subcortical volumes subcortical receptomes 1, 2 and 3 (in order).\n"

# Save the correlation_data dictionary to the text file
with open(output_file, 'w') as file:
    file.write(header_text)
    json.dump(correlation_data, file)

print(sxc_1)
print(sxc_2)
print(sxc_3)


#Correlation paddington-gradients
sxc_1,p1 = stats.permtest_pearsonr(paddington_sxc.values, s_g1_df.values)
sxc_2,p2 = stats.permtest_pearsonr(paddington_sxc.values, s_g2_df.values)
sxc_3,p3 = stats.permtest_pearsonr(paddington_sxc.values, s_g3_df.values)

p_values = [p1, p2, p3]

# Perform FDR correction
corrected_p_values = multipletests(p_values, method='fdr_bh')[1]

# Assign the corrected p-values back to p1, p2, and p3
p1, p2, p3 = corrected_p_values

correlation_data = {
    "p1": p1,
    "sxc_1": sxc_1,
    "p2": p2,
    "sxc_2": sxc_2,
    "p3": p3,
    "sxc_3": sxc_3
}

output_file = "/Users/charlie/Desktop/my_projects/neurotransmitter/receptome/results/permut_sxc_paddington.txt"

header_text = "Permuted correlation coefficients/Pvals in Track between voxelwise subcortical volumes subcortical receptomes 1, 2 and 3 (in order).\n"

# Save the correlation_data dictionary to the text file
with open(output_file, 'w') as file:
    file.write(header_text)
    json.dump(correlation_data, file)

print(sxc_1)
print(sxc_2)
print(sxc_3)


